# Introduction

This notebook demonstrates basic usage of BioThings Explorer, an engine for autonomously querying a distributed knowledge graph. BioThings Explorer can answer two classes of queries -- "EXPLAIN" and "PREDICT".  EXPLAIN queries are described in [EXPLAIN_demo.ipynb](EXPLAIN_demo.ipynb). Here, we describe PREDICT queries and how to use BioThings Explorer to execute them.  A more detailed overview of the BioThings Explorer systems is provided in [these slides](https://docs.google.com/presentation/d/1QWQqqQhPD_pzKryh6Wijm4YQswv8pAjleVORCPyJyDE/edit?usp=sharing).

PREDICT queries are designed to **predict plausible relationships between one entity and an entity class**.  For example, in this notebook, we explore the question:

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;*"What drugs might be used to treat hyperphenylalaninemia?"*

**To experiment with an executable version of this notebook, [load it in Google Colaboratory](https://colab.research.google.com/github/biothings/biothings_explorer/blob/master/jupyter%20notebooks/PREDICT_demo.ipynb).**

## Step 0: Load BioThings Explorer modules

First, install the `biothings_explorer` and `biothings_schema` packages, as described in this [README](https://github.com/biothings/biothings_explorer/blob/master/jupyter%20notebooks/README.md#prerequisite).  This only needs to be done once (but including it here for compability with [colab](https://colab.research.google.com/)).

In [1]:
!pip install git+https://github.com/biothings/biothings_explorer#egg=biothings_explorer

You should consider upgrading via the 'pip install --upgrade pip' command.


Next, import the relevant modules:

* **Hint**: Find corresponding bio-entity representation used in BioThings Explorer based on user input (could be any database IDs, symbols, names)
* **FindConnection**: Find intermediate bio-entities which connects user specified input and output

In [2]:
# import modules from biothings_explorer
from biothings_explorer.hint import Hint
from biothings_explorer.user_query_dispatcher import FindConnection

## Step 1: Find representation of "hyperphenylalaninemia" in BTE

In this step, BioThings Explorer translates our query string "hyperphenylalaninemia"  into BioThings objects, which contain mappings to many common identifiers.  Generally, the top result returned by the `Hint` module will be the correct item, but you should confirm that using the identifiers shown.

Search terms can correspond to any child of [BiologicalEntity](https://biolink.github.io/biolink-model/docs/BiologicalEntity.html) from the [Biolink Model](https://biolink.github.io/biolink-model/docs/), including `DiseaseOrPhenotypicFeature` (e.g., "lupus"), `ChemicalSubstance` (e.g., "acetaminophen"), `Gene` (e.g., "CDK2"), `BiologicalProcess` (e.g., "T cell differentiation"), and `Pathway` (e.g., "Citric acid cycle").

In [3]:
ht = Hint()
# find all potential representations of hyperphenylalaninemia
hyperphenylalaninemia_hint = ht.query("hyperphenylalaninemia")
# select the correct representation of hyperphenylalaninemia
hyperphenylalaninemia = hyperphenylalaninemia_hint['DiseaseOrPhenotypicFeature'][0]
hyperphenylalaninemia

{'mondo': 'MONDO:0016543',
 'umls': 'C0751435',
 'name': 'hyperphenylalaninemia',
 'display': 'mondo(MONDO:0016543) umls(C0751435) name(hyperphenylalaninemia) ',
 'type': 'DiseaseOrPhenotypicFeature',
 'primary': {'identifier': 'mondo',
  'cls': 'DiseaseOrPhenotypicFeature',
  'value': 'MONDO:0016543'}}

## Step 2: Find drugs that are associated with genes which invovled in hyperphenylalaninemia

In this section, we find all paths in the knowledge graph that connect hyperphenylalaninemia to any entity that is a chemical compound.  To do that, we will use `FindConnection`.  This class is a convenient wrapper around two advanced functions for **query path planning** and **query path execution**. More advanced features for both query path planning and query path execution are in development and will be documented in the coming months. 

The parameters for `FindConnection` are described below:


In [4]:
help(FindConnection.__init__)

Help on function __init__ in module biothings_explorer.user_query_dispatcher:

__init__(self, input_obj, output_obj, intermediate_nodes, registry=None)
    Find relationships in the Knowledge Graph between an Input Object and an Output Object.
    
    Args:
        input_obj (required): must be an object returned from Hint corresponding to a specific biomedical entity.
                            Examples: 
                Hint().query("Fanconi anemia")['DiseaseOrPhenotypicFeature'][0]
                Hint().query("acetaminophen")['ChemicalSubstance'][0]
    
        output_obj (required): must EITHER be an object returned from Hint corresponding to a specific biomedical
                            entity, OR be a string or list of strings corresponding to Biolink Entity classes.
                            Examples:
                Hint().query("acetaminophen")['ChemicalSubstance'][0]
                'Gene'
                ['Gene','ChemicalSubstance']
    
        intermediate_nodes 

Here, we formulate a `FindConnection` query with "hyperphenylalaninemia" as the `input_ojb`, "ChemicalSubstance" as the `output_obj` (which corresponds to a Biolink Entity type).  We further specify with the `intermediate_nodes` parameter that we are looking for paths joining hyperphenylalaninemia and chemical compounds with *one* intermediate node that is a Gene.  (The ability to search for longer reasoning paths that include additional intermediate nodes will be added shortly.)

In [5]:
fc = FindConnection(input_obj=hyperphenylalaninemia, output_obj='ChemicalSubstance', intermediate_nodes=['Gene'])

We next execute the `connect` method, which performs the **query path planning** and **query path execution** process.  In short, BioThings Explorer is deconstructing the query into individual API calls, executing those API calls, then assembling the results.

A verbose log of this process is displayed below:

In [6]:
# set verbose=True will display all steps which BTE takes to find the connection
fc.connect(verbose=True)

========== QUERY PARAMETER SUMMARY ==========

BTE will find paths that join 'hyperphenylalaninemia' and 'ChemicalSubstance'. Paths will have 1 intermediate node.

Intermediate node #1 will have these type constraints: Gene



========== QUERY #1 -- fetch all Gene entities linked to hyperphenylalaninemia ==========

==== Step #1: Query path planning ====

Because hyperphenylalaninemia is of type 'DiseaseOrPhenotypicFeature', BTE will query our meta-KG for APIs that can take 'DiseaseOrPhenotypicFeature' as input and 'Gene' as output

BTE found 4 apis:

API 1. biolink_disease2gene(1 API call)
API 2. DISEASES(1 API call)
API 3. mydisease.info(1 API call)
API 4. semmeddisease(1 API call)


==== Step #2: Query path execution ====
NOTE: API requests are dispatched in parallel, so the list of APIs below is ordered by query time.

API 3.1: http://mydisease.info/v1/query (POST "q=C0751435,C0751436&scopes=mondo.xrefs.umls,disgenet.xrefs.umls&fields=disgenet.genes_related_to_disease&species=human

API 3.11 robokop_gene2chemical: 3 hits
API 3.12 robokop_gene2chemical: 3 hits
API 5.1 semmedgene: 5497 hits

After id-to-object translation, BTE retrieved 2891 unique objects.

========== Final assembly of results ==========


In the #1 query, BTE found 17 unique Gene nodes
In the #2 query, BTE found 2891 unique ChemicalSubstance nodes


## Step 3: Display and Filter results
This section demonstrates post-query filtering done in Python. Later, more advanced filtering functions will be added to the **query path execution** module for interleaved filtering, thereby enabling longer query paths. More details to come...

First, all matching paths can be exported to a data frame. Let's examine a sample of those results.

In [7]:
df = fc.display_table_view()

# because UMLS is not currently well-integrated in our ID-to-object translation system, removing UMLS-only outputs here
patternDel = "^umls:C\d+"
filter = df.output_id.str.contains(patternDel)
df = df[~filter]

df.sample(10)

,input,input_type,pred1,pred1_source,pred1_api,pred1_pubmed,node1_type,node1_name,node1_id,pred2,pred2_source,pred2_api,pred2_pubmed,output_type,output_name,output_id
9078,HYPERPHENYLALANINEMIA,DiseaseOrPhenotypicFeature,associatedWith,biolink,biolink_disease2gene,None,Gene,symbol:QDPR,symbol:QDPR,targetedBy,mychem.info,mychem.info,None,ChemicalSubstance,drugbank:DB03814,drugbank:DB03814
2623,HYPERPHENYLALANINEMIA,DiseaseOrPhenotypicFeature,associatedWith,semmed,semmeddisease,27612877,Gene,symbol:COG2,symbol:COG2,molecularlyInteractsWith,semmed,semmedgene,18640465,ChemicalSubstance,PITAVASTATIN,name:PITAVASTATIN
3405,HYPERPHENYLALANINEMIA,DiseaseOrPhenotypicFeature,associatedWith,semmed,semmeddisease,27612877,Gene,symbol:COG2,symbol:COG2,molecularlyInteractsWith,semmed,semmedgene,"11406909,11376309,12648030,9627539,17577472,15...",ChemicalSubstance,ATORVASTATIN,name:ATORVASTATIN
8642,HYPERPHENYLALANINEMIA,DiseaseOrPhenotypicFeature,associatedWith,disgenet,mydisease.info,None,Gene,symbol:PAH,symbol:PAH,molecularlyInteractsWith,semmed,semmedgene,19905008,ChemicalSubstance,LACTIC ACID,name:LACTIC ACID
8484,HYPERPHENYLALANINEMIA,DiseaseOrPhenotypicFeature,causedBy,semmed,semmeddisease,"9222755,9667588",Gene,umls:C0017337,umls:C0017337,molecularlyInteractsWith,semmed,semmedgene,7737473,ChemicalSubstance,OLEANDOMYCIN,name:OLEANDOMYCIN
3312,HYPERPHENYLALANINEMIA,DiseaseOrPhenotypicFeature,associatedWith,semmed,semmeddisease,16917891,Gene,umls:C0002085,umls:C0002085,molecularlyInteractsWith,semmed,semmedgene,12937870,ChemicalSubstance,MEXILETINE,name:MEXILETINE
3346,HYPERPHENYLALANINEMIA,DiseaseOrPhenotypicFeature,associatedWith,semmed,semmeddisease,16917891,Gene,umls:C0002085,umls:C0002085,molecularlyInteractsWith,semmed,semmedgene,"28023164,28021968",ChemicalSubstance,TAMOXIFEN,name:TAMOXIFEN
8526,HYPERPHENYLALANINEMIA,DiseaseOrPhenotypicFeature,causedBy,semmed,semmeddisease,"9222755,9667588",Gene,umls:C0017337,umls:C0017337,molecularlyInteractsWith,semmed,semmedgene,19689802,ChemicalSubstance,SALICYLIC ACID,name:SALICYLIC ACID
8465,HYPERPHENYLALANINEMIA,DiseaseOrPhenotypicFeature,associatedWith,disgenet,mydisease.info,None,Gene,symbol:PAH,symbol:PAH,molecularlyInteractsWith,semmed,semmedgene,2226440,ChemicalSubstance,NOREPINEPHRINE,name:NOREPINEPHRINE
8190,HYPERPHENYLALANINEMIA,DiseaseOrPhenotypicFeature,causedBy,semmed,semmeddisease,"9222755,9667588",Gene,umls:C0017337,umls:C0017337,molecularlyInteractsWith,semmed,semmedgene,18595847,ChemicalSubstance,CIANIDANOL,name:CIANIDANOL


While most results are based on edges from [semmed](https://skr3.nlm.nih.gov/SemMed/), edges from [DGIdb](http://www.dgidb.org/), [biolink](https://monarchinitiative.org/), [disgenet](http://www.disgenet.org/), and [mychem.info](https://mychem.info) were also retrieved from their respective APIs. 

Next, let's examine which chemical compounds were most frequently mentioned.

In [8]:
df.output_name.value_counts().head(10)

DEXTROSE           25
ALCOHOL            20
NOREPINEPHRINE     19
HYDROCORTISONE     17
DOPAMINE           17
SODIUM CHLORIDE    17
SAPROPTERIN        15
QUERCETIN          15
VITAMIN E          14
NITRIC OXIDE       14
Name: output_name, dtype: int64

Hyperphenylalaninemia is a condition characterized by elevated levels of phenylalanine in the blood. This phenotype is strongly associated with phenylketonuria (PKU), an inherited, genetic disorder that affects the ability to metabolize phenylalanine.  **Sapropterin** is a naturally-occurring cofactor associated with several enzymatic processes, including the metabolism of phenylalanine to tyrosine. It has been [FDA-approved](https://www.centerwatch.com/drug-information/fda-approved-drugs/drug/972/kuvan-sapropterin-dihydrochloride) as a treatment for PKU patients.  Tyrosine is also a precursor to several neurotransmitters, including **norepinephrine** and **dopamine**.

While several of the listed compounds would be considered positive controls, others on the list could be viewed as **testable hypotheses and discovery opportunities** to be evaluated by domain experts. For example, let's examine which genes link hyperphenylalaninemia to nitric oxide.

In [9]:
df[df.output_name=="NITRIC OXIDE"]

,input,input_type,pred1,pred1_source,pred1_api,pred1_pubmed,node1_type,node1_name,node1_id,pred2,pred2_source,pred2_api,pred2_pubmed,output_type,output_name,output_id
2472,HYPERPHENYLALANINEMIA,DiseaseOrPhenotypicFeature,associatedWith,semmed,semmeddisease,25260610,Gene,umls:C1705543,umls:C1705543,molecularlyInteractsWith,semmed,semmedgene,"9388488,25706765",ChemicalSubstance,NITRIC OXIDE,name:NITRIC OXIDE
2473,HYPERPHENYLALANINEMIA,DiseaseOrPhenotypicFeature,associatedWith,semmed,semmeddisease,25260610,Gene,umls:C1705543,umls:C1705543,molecularlyInteractsWith,semmed,semmedgene,"19779140,22384120,26665235",ChemicalSubstance,NITRIC OXIDE,name:NITRIC OXIDE
2474,HYPERPHENYLALANINEMIA,DiseaseOrPhenotypicFeature,associatedWith,semmed,semmeddisease,9675155,Gene,umls:C0017337,umls:C0017337,molecularlyInteractsWith,semmed,semmedgene,"8720483,8874195,9220344,8642187,1665333,104325...",ChemicalSubstance,NITRIC OXIDE,name:NITRIC OXIDE
2475,HYPERPHENYLALANINEMIA,DiseaseOrPhenotypicFeature,causedBy,semmed,semmeddisease,"9222755,9667588",Gene,umls:C0017337,umls:C0017337,molecularlyInteractsWith,semmed,semmedgene,"8720483,8874195,9220344,8642187,1665333,104325...",ChemicalSubstance,NITRIC OXIDE,name:NITRIC OXIDE
2476,HYPERPHENYLALANINEMIA,DiseaseOrPhenotypicFeature,associatedWith,semmed,semmeddisease,9675155,Gene,umls:C0017337,umls:C0017337,molecularlyInteractsWith,semmed,semmedgene,"8874195,7945348,12687343,12381414,9853913,9853...",ChemicalSubstance,NITRIC OXIDE,name:NITRIC OXIDE
2477,HYPERPHENYLALANINEMIA,DiseaseOrPhenotypicFeature,causedBy,semmed,semmeddisease,"9222755,9667588",Gene,umls:C0017337,umls:C0017337,molecularlyInteractsWith,semmed,semmedgene,"8874195,7945348,12687343,12381414,9853913,9853...",ChemicalSubstance,NITRIC OXIDE,name:NITRIC OXIDE
2478,HYPERPHENYLALANINEMIA,DiseaseOrPhenotypicFeature,associatedWith,biolink,biolink_disease2gene,None,Gene,symbol:GCH1,symbol:GCH1,molecularlyInteractsWith,semmed,semmedgene,18671723,ChemicalSubstance,NITRIC OXIDE,name:NITRIC OXIDE
2479,HYPERPHENYLALANINEMIA,DiseaseOrPhenotypicFeature,associatedWith,disgenet,mydisease.info,None,Gene,symbol:GCH1,symbol:GCH1,molecularlyInteractsWith,semmed,semmedgene,18671723,ChemicalSubstance,NITRIC OXIDE,name:NITRIC OXIDE
2481,HYPERPHENYLALANINEMIA,DiseaseOrPhenotypicFeature,associatedWith,biolink,biolink_disease2gene,None,Gene,symbol:GCH1,symbol:GCH1,molecularlyInteractsWith,semmed,semmedgene,"20870896,19447893",ChemicalSubstance,NITRIC OXIDE,name:NITRIC OXIDE
2482,HYPERPHENYLALANINEMIA,DiseaseOrPhenotypicFeature,associatedWith,disgenet,mydisease.info,None,Gene,symbol:GCH1,symbol:GCH1,molecularlyInteractsWith,semmed,semmedgene,"20870896,19447893",ChemicalSubstance,NITRIC OXIDE,name:NITRIC OXIDE


BTE identifies the genes [HSPA4](https://www.ncbi.nlm.nih.gov/gene/3308) and [GCH1](https://www.ncbi.nlm.nih.gov/gene/2643) as being the key nodes that join hyperphenylalaninemia with nitric oxide.

## Conclusions and caveats

This notebook demonstrated the use of BioThings Explorer in PREDICDT mode to identify chemical compounds related to hyperphenylalaninemia. BioThings Explorer autonomously queried a **distributed knowledge graph of biomedical APIs** to find the most common compounds, and several of the top hits had a clear biochemical and clinical relevance to hyperphenylalaninemia.

There are still many areas for improvement (and some areas in which BioThings Explorer is still buggy).  And of course, BioThings Explorer is dependent on the accessibility of the APIs that comprise the distributed knowledge graph.  Nevertheless, we encourage users to try other variants of the PREDICT queries demonstrated in this notebook.